## Enunciado

>Considere el proyecto cuya información se adjunta.
> 1. Determine para cada actividad su duración optimista. ¿Influye esta duración optimista en la duración mínima del proyecto?
> 2. Determine duración media del proyecto y su varianza utilizando para ello la aproximación del teorema del límite central.
> 3. Determine la probabilidad de que el proyecto se termine antes de 10 periodos.
> 4. Planifique el proyecto teniendo en cuenta que durante las primeras tres semanas dispondrá de cuatro trabajadores y a partir de la cuarta de seis trabajadores.
> 5. ¿Sería suficiente disponer de 4 recursos durante todo el proyecto? ¿Por qué?



In [1]:
!apt install libgraphviz-dev              > /dev/null
%pip install pygraphviz "pandas[excel]"   > /dev/null
%pip install numpyarray_to_latex          > /dev/null
%pip install -q amplpy                    > /dev/null

repo_url = 'https://github.com/mcasl/pmULE'
repo_name = "pmULE"

import os, sys

if not os.path.exists(repo_name):
    print(f"Cloning repository '{repo_name}'...")
    !git clone {repo_url}
    print("Repository cloned successfully!")
else:
    print(f"Good! No need to download '{repo_name}' again.")

from google.colab import files

# To find local version of the library
sys.path.append('pmULE/src')

from pmule import PredecessorTable, ProjectGraph

zsh:1: command not found: apt
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Cloning repository 'pmULE'...
Cloning into 'pmULE'...
remote: Enumerating objects: 329, done.
remote: Counting objects: 100% (138/138), done.
remote: Compressing objects: 100% (135/135), done.
remote: Total 329 (delta 88), reused 7 (delta 2), pack-reused 191 (from 1)
Receiving objects: 100% (329/329), 10.51 MiB | 16.48 MiB/s, done.
Resolving deltas: 100% (189/189), done.
Repository cloned successfully!


ModuleNotFoundError: No module named 'google'

In [ ]:
# Alargamos el ancho de las celdas al 90%
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

import pandas as pd
import numpy as np

datos_enunciado = pd.DataFrame([
#  nombre, anteriores, duracion, duracion_pesimista, duracion_modal, recursos
     ('A',       '---  ',     2,         2,                    2,           1  ),
     ('B',       'A    ',     2,         3,                    2,           2  ),
     ('C',       'B    ',     3,         4,                    3,           2  ),
     ('D',       'B,J  ',     4,         5,                 17/4,           1  ),
     ('E',       'F,B,J',     1,         1,                    1,           1  ),
     ('F',       'A    ',     2,         3,                    2,           2  ),
     ('G',       'C,D  ',     1,         1,                    1,           1  ),
     ('H',       'E    ',     4,         5,                    4,           1  ),
     ('I',       'B,J  ',     1,         1,                    1,           1  ),
     ('J',       'A    ',     2,         2,                    2,           1  ),
] ,columns=['activity', 'predecessor', 'duration', 'pesimistic_duration', 'modal_duration', 'resources'])
datos_enunciado.set_index('activity', inplace=True)
datos_enunciado

## Solución
### Apartado 1

> Determine para cada actividad su duración optimista. ¿Influye esta duración optimista en la duración tope del proyecto?


A partir de los datos del enunciado y de la relación $D=\frac{a+4m+b}{6}$ obtenemos la duración optimista $a=6D-b-4m$.

In [ ]:
datos_proyecto = datos_enunciado.copy()
datos_proyecto['optimistic_duration'] = datos_proyecto.eval('6*duration - pesimistic_duration - 4*modal_duration')
datos_proyecto

### Apartado 2
> Determine duración media del proyecto y su varianza utilizando para ello la aproximación del teorema del límite central.


#### Duración media del proyecto

In [ ]:
prelaciones = PredecessorTable.from_dataframe_of_strings(datos_proyecto, activity='activity', predecessor='predecessor')
prelaciones.display_immediate_linkage_matrix()

La clase PertGraph permite visualizar el grafo PERT, el diagrama de Gantt así como realizar el cálculo de tiempos tempranos, tardíos y de las holguras totales.

La varianza de cada actividad no es aportada por el enunciado como dato, pero sí se puede calcular a partir de la expresión $\sigma^2 = \left(\frac{b-a}{6} \right)$

In [ ]:
datos_proyecto['variance'] = datos_proyecto.eval('((pesimistic_duration - optimistic_duration)/6)**2')
datos_proyecto[['pesimistic_duration', 'optimistic_duration', 'variance']]

##### Grafo PERT con numeración de nodos

In [ ]:
proyecto = prelaciones.create_project()
proyecto.pert(ranksep=1.0)

##### Matriz de Zaderenko

In [ ]:
proyecto.zaderenko(durations=datos_proyecto['duration'])

In [ ]:
duraciones_medias = datos_proyecto['duration']
resultados_pert = proyecto.calculate_pert(durations=duraciones_medias)
resultados_pert['nodes']

La duración media del proyecto es:

In [ ]:
duracion_media_proyecto = proyecto.duration(durations=duraciones_medias)
duracion_media_proyecto

#### Varianza de la duración del proyecto
Para determinar la varianza de la duración del proyecto es necesario determinar la varianza de cada rama del camino crítico.

##### Holguras

In [ ]:
resultados_pert['activities'][['H_total']]

##### Camino crítico

##### Grafo PERT con indicación del camino crítico

In [ ]:
proyecto.pert(durations=duraciones_medias, ranksep=1.5)

In [ ]:
camino_critico = proyecto.critical_path(durations=duraciones_medias)
camino_critico

##### Varianza del proyecto

In [ ]:
desviacion_proyecto = proyecto.standard_deviation(durations=datos_proyecto['duration'],
                                                  variances=datos_proyecto['variance'])
desviacion_proyecto

### Apartado 3
> Determine la probabilidad de que el proyecto se termine antes de 10 periodos.



In [ ]:
import scipy
from scipy.stats import norm

probabilidad_de_terminar_antes_de_10 = norm.cdf(10, loc=duracion_media_proyecto, scale=desviacion_proyecto)
probabilidad_de_terminar_antes_de_10 * 100

### Apartado 4
> Planifique el proyecto teniendo en cuenta que durante las primeras tres semanas dispondrá de cuatro trabajadores y a partir de la cuarta de seis trabajadores.


#### Análisis de la situación inicial

Para responder este apartado es necesario representar la demanda de recursos a lo largo del tiempo.

In [ ]:
proyecto_asignacion = proyecto.copy()
gantt_asignacion = proyecto_asignacion.gantt(datos_proyecto, 'duration', 'resources', total='fila', holguras = True)
gantt_asignacion

In [ ]:
(gantt_asignacion.data.loc['Total'].drop('H_total').plot(kind='bar'));

La demanda de recursos actual infringe el límite impuesto en el enunciado. Es necesario aplicar el algoritmo de asignación.

#### Reprogramación del proyecto utilizando el algoritmo de asignación

 El primer periodo problemático es el 3.

##### Asignación periodo 3

Las tres actividades programadas para el periodo 3 son críticas y de la misma duración. Al disponer de 4 recursos máximo se programan las actividades B y F desplazando la J.

In [ ]:
duracion_y_recursos = datos_proyecto[['duration', 'resources']].copy()
duracion_y_recursos = proyecto_asignacion.desplazar(duracion_y_recursos, 'duration', 'resources', J = 1)

La actividad *💤J* representa el desplazamiento de las actividad J.

En estas condiciones el proyecto ya es compatible con las restricciones impuestas a los recursos.

### Apartado 5
> ¿Sería suficiente disponer de 4 recursos durante todo el proyecto? ¿Por qué?

En la programación resultante del apartado anterior los cuatro recursos no serían suficientes pues hay periodos que demandan 5 recursos. Se analiza la posibilidad de reprogramar el proyecto atendiendo a este nuevo límite.

#### Reprogramación del proyecto utilizando el algoritmo de asignación

 El primer periodo problemático es el 4.

##### Asignación periodo 4

Las tres actividades programadas para el periodo 4 son críticas y de la misma duración. Sin embargo las actividades y B y F ya han comenzado su ejecución y se opta por no interrumpirlas. Se desplaza por tanto la J.

In [ ]:
duracion_y_recursos = proyecto_asignacion.desplazar(duracion_y_recursos, 'duration', 'resources', J = 1)

El siguiente periodo conflictivo sería el 7.

##### Asignación periodo 7

Las D y E son críticas. Entre ambas consumen dos recursos. De las dos actividades no críticas se opta por desplazar la I por ser la de mayor holgura. La actividad C es posible programarla para este periodo al consumir otros dos recurssos.

In [ ]:
duracion_y_recursos = proyecto_asignacion.desplazar(duracion_y_recursos, 'duration', 'resources', I = 1)

En estas condiciones el proyecto ya es viable con 4 recursos. No obstante su duración se ha prolongado hasta 11 periodos.